# Whole brain data processing

In [1]:
import numpy as np
import pandas as pd
import os, sys
from glob import glob
from h5py import File
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2
from fish_proc.utils.memory import get_process_memory, clear_variables
cameraNoiseMat = '/groups/ahrens/ahrenslab/Ziqiang/gainMat/gainMat20180208'
dir_root = '/nrs/ahrens/Yu/SPIM/active_dataset/glia_neuron_imaging/20161109/fish2/20161109_2_1_6dpf_GFAP_GC_Huc_RG_GA_CL_fb_OL_f0_0GAIN_20161109_211950/raw'
import warnings
warnings.filterwarnings('ignore')
import dask.array as da
from utils import *
import zarr

## Set up dask

In [2]:
import fish_proc.utils.dask_ as fdask
cluster, client = fdask.setup_workers(60)
client

Client Scheduler: tcp://10.36.111.11:38130 Dashboard: http://10.36.111.11:39010/status,Cluster Workers: 5 Cores: 5 Memory: 80.00 GB


## Load file to dask

In [3]:
files = sorted(glob(dir_root+'/*.h5'))
chunks = File(files[0],'r')['default'].shape
files = files[::10] # this is for test purpose
data = da.stack([da.from_array(File(fn,'r')['default'], chunks=chunks) for fn in files])

## Pixel denoise
* Test of comparison -- see notebook test_data_compression

In [4]:
from fish_proc.utils.getCameraInfo import getCameraInfo
cameraInfo = getCameraInfo(dir_root)
denoised_data = data.map_blocks(lambda v: pixelDenoiseImag(v, cameraInfo=cameraInfo), dtype='float32')

In [13]:
if not os.path.exists('motion_fix_.h5'):
    med_win = len(denoised_data)
    ref_img = denoised_data[med_win-100:med_win+100].mean(axis=0).compute()
    save_h5('motion_fix_.h5', ref_img, dtype='float16')
else:
    ref_img = File('motion_fix_.h5', 'r')['default'].value

## Registration

In [ ]:
%%time
if not os.path.exists('trans_affs.npy'):
    trans_affine = denoised_data.map_blocks(lambda x: estimate_rigid2d(x, fixed=ref_img), dtype='float32', drop_axis=(3), chunks=(1,4,4)).compute()
    np.save('trans_affs', trans_affine)

In [ ]:
trans_affine_ = np.load('trans_affs.npy')
trans_affine_ = da.from_array(trans_affine_, chunks=(1,4,4))

In [ ]:
plt.plot(trans_affine_[:, 1, -1])
plt.plot(trans_affine_[:, 2, -1])
plt.show()

### Prepare data for video detrend

In [ ]:
trans_data_ = da.map_blocks(apply_transform3d, denoised_data, trans_affine_, chunks=(1, *denoised_data.shape[1:]), dtype='float32')
trans_data_t = trans_data_.transpose((1, 2, 3, 0)).rechunk((1, 'auto', 'auto', -1))

In [ ]:
%%time
import shutil
if os.path.exists('registered_data'):
    shutil.rmtree('registered_data')
trans_data_t.to_zarr('registered_data')

In [32]:
cluster.stop_all_jobs()

### Save registered image
`trans_data_t` will be used for df/f computation after spatal component matrix is obtained

## Video Detrend
* use more trackable video detrend algorithm -- for example running percentile instead of local fit -- this will saving space in trend data

In [ ]:
trans_data_t = da.from_zarr('registered_data')

In [ ]:
trans_data_t

In [ ]:
def detrend(mov):
    from fish_proc.denoiseLocalPCA.detrend import trend
    return mov - trend(mov)

In [ ]:
detrend_data = trans_data_t.map_blocks

## Local PCA
* This data should be saved as the sparse matrices

## Demix